In [1]:
import tifffile as tiff
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact
import time
from IPython.display import clear_output

%matplotlib inline
interactive = True
recompute_bounds = False
plt.rcParams['figure.dpi'] = 150

In [2]:
baseline = tiff.imread('../data/baseline.tiff')
detrended = tiff.imread('../data/detrended.tiff')

# fast enough for a one-time computation
if recompute_bounds:
    lower = min([min(baseline.flatten() + detrended[i].flatten()) for i in range(detrended.shape[0])])
    upper = max([max(baseline.flatten() + detrended[i].flatten()) for i in range(detrended.shape[0])])
else:
    lower, upper = 16646655, 33293310

In [3]:
def plot_river_elevation(time, elevation, azimut, dim):
    # create data points
    x = range(baseline.shape[1])
    y = range(baseline.shape[0])
    # create grid
    X, Y = np.meshgrid(x, y)
    Z = baseline + detrended[time]
    
    # figure
    fig = plt.figure(figsize=(8, 8))
    #fig.suptitle("River bed elevation")
    
    if dim == 3:
        # 3d plot
        ax = fig.add_subplot(1, 1, 1, projection='3d')
        ax.view_init(elevation, azimut)
        ax.plot_surface(X, Y, Z, cmap='viridis')
        ax.set_zlim3d(lower, upper)
        ax.set_box_aspect((10, 1, 1))
        ax.autoscale_view('tight')
        ax.margins(0)
        ax.get_xaxis().set_ticks([])
        ax.get_yaxis().set_ticks([])
        ax.get_zaxis().set_ticks([])
    else:
        # 2d plot
        fig.add_subplot(1, 1, 1)
        image = plt.imshow(Z, origin='lower')
    
    # show
    plt.show()
    return fig

@interact(time=(0, detrended.shape[0] - 1, 1), elevation=(-90,90,5), azimut=(-90,90,5), dimensions=[('2D', 2), ('3D', 3)])
def interact_plot(time=0, elevation=90, azimut=-90, dimensions=2):
    plot_river_elevation(time, elevation, azimut, dimensions)
    


interactive(children=(IntSlider(value=0, description='time', max=661), IntSlider(value=90, description='elevat…

In [5]:
@interact
def animate():
    t = 0
    step = 5
    while t < detrended.shape[0]:
        plot_river_elevation(t, 50, -65, 3)
        clear_output(wait=True)
        t += step
    plot_river_elevation(detrended.shape[0] - 1, 50, -65, 3)

interactive(children=(Output(),), _dom_classes=('widget-interact',))